In [27]:
import os, operator, nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import networkx as nx
import codecs

In [32]:
dataset_dir = 'Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/'
    
for subdir, dirs, files in os.walk(dataset_dir):
    # iterate over all files in train/test directory
    for file in files:
        if not file.endswith('.txt'):
            continue
#         f = open(subdir+'/'+file, 'r', encoding='utf-8')
        f = codecs.open(subdir+'/'+file, "r", "utf-8" )
        text = f.read()
#         print file
        # tokenizing the text and filtering nouns / verbs etc
        tokens = nltk.word_tokenize(text)
        tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

        def filter_words(key):
            return True if key.startswith('NN') or key.startswith('J') or key.startswith('VB') else False
        filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))
        
        # creating local maps for nodes and edges
        dict_text = defaultdict(int)
        for t in filt_text:
            dict_text[t] += 1
        edges, nodes, nodes_number = [], {}, {}
        c = 0
        for d in dict_text:
            if d in nodes:
                continue
            nodes[d] = c
            nodes_number[c] = d
            c += 1
            
        # create edges for graph-of-words
        k = 10 ## window size
        for i in range(len(filt_text)-k):
            for j in range(i+1, i+k+1):
                wt = j-i  ## this is the weight of the edge (distance between words)
                edges.append((nodes[filt_text[i]], nodes[filt_text[j]], wt))
print "Done!"
        # TODO XX_Jahnavi: Write these edges to a file

Done!


In [33]:
f = open('Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/train/art_and_culture-20893614.txt', 'r')
text = f.read()

In [34]:
# tokenizing the text and filtering nouns / verbs etc
tokens = nltk.word_tokenize(text)
tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

def filter_words(key):
    return True if key.startswith('NN') or key.startswith('J') or key.startswith('VB') else False
filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))

In [35]:
# creating local maps for nodes and edges
dict_text = defaultdict(int)
for t in filt_text:
    dict_text[t] += 1
edges = []
nodes = {}
nodes_number = {}
c = 0
for d in dict_text:
    if d in nodes:
        continue
    nodes[d] = c
    nodes_number[c] = d
    c += 1

In [36]:
# create edges for graph-of-words
k = 5 ## window size
for i in range(len(filt_text)-k):
    for j in range(i+1, i+k+1):
        wt = j-i  ## this is the weight of the edge (distance between words)
        edges.append((nodes[filt_text[i]], nodes[filt_text[j]], wt))

In [37]:
# create graph-of-words
G=nx.Graph()

### TODO: read from file to add edges
G.add_weighted_edges_from(edges)

### Priority Queue implementation used by IC

In [38]:
import itertools
from heapq import *

class PriorityQueue(object):
    def __init__(self):
        self.pq = []                         # list of entries arranged in a heap
        self.entry_finder = {}               # mapping of tasks to entries
        self.REMOVED = '<removed-task>'      # placeholder for a removed task
        self.counter = itertools.count()     # unique sequence count

    def add_task(self, task, priority=0):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove_task(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def remove_task(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED

    def pop_item(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.pq:
            priority, count, task = heappop(self.pq)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task, priority
        raise KeyError('pop from an empty priority queue')

    def __str__(self):
        return str([entry for entry in self.pq if entry[2] != self.REMOVED])

### IC function

In [39]:
def runIC (G, S, p = .01):
    ''' Runs independent cascade model.
    Input: G -- networkx graph object
    S -- initial set of vertices
    p -- propagation probability
    Output: T -- resulted influenced set of vertices (including S)
    '''
    from copy import deepcopy
    from random import random
    T = deepcopy(S) # copy already selected nodes

    # ugly C++ version
    i = 0
    while i < len(T):
        for v in G[T[i]]: # for neighbors of a selected node
            if v not in T: # if it wasn't selected yet
                w = G[T[i]][v]['weight'] # count the number of edges between two nodes
                if random() <= 1 - (1-p)**w: # if at least one of edges propagate influence
                    # print T[i], 'influences', v
                    T.append(v)
        i += 1

    # neat pythonic version
    # legitimate version with dynamically changing list: http://stackoverflow.com/a/15725492/2069858
    # for u in T: # T may increase size during iterations
    #     for v in G[u]: # check whether new node v is influenced by chosen node u
    #         w = G[u][v]['weight']
    #         if v not in T and random() < 1 - (1-p)**w:
    #             T.append(v)
    return T

### General Greedy algo from Kempe paper

In [51]:
# from priorityQueue import PriorityQueue as PQ
# from IC.IC import runIC

def generalGreedy(G, k, p=.01):
    ''' Finds initial seed set S using general greedy heuristic
    Input: G -- networkx Graph object
    k -- number of initial nodes needed
    p -- propagation probability
    Output: S -- initial set of k nodes to propagate
    '''
    import time
    start = time.time()
    R = 5 # number of times to run Random Cascade
    S = [] # set of selected nodes
    # add node to S if achieves maximum propagation for current chosen + this node
    for i in range(k):
        print i, " ",
        s = PQ() # priority queue
        for v in G.nodes():
            if v not in S:
                s.add_task(v, 0) # initialize spread value
                for j in range(R): # run R times Random Cascade
                    [priority, count, task] = s.entry_finder[v]
                    s.add_task(v, priority - float(len(runIC(G, S + [v], p)))/R) # add normalized spread value
        task, priority = s.pop_item()
        S.append(task)
#         print i, k, time.time() - start
    return S

In [ ]:
result = generalGreedy(G, 58)

0   1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33  

In [48]:
prediction = []
for node_num in result:
    prediction.append(nodes_number[node_num])
    print nodes_number[node_num]

sitcoms
Lopilato
sweetheart
traditional
civil
Facebook
homes
day
threw
album
followers
years
Grammy
dress
Buenos
woman
guests
Canada
gray
star
Children
top-grossing
Monaco
British
love
Girls
including
BUENOS
high
wedding
pop
model
several
sharp
Michael
Soul
young
TV
media
AIRES
shouted
American
Pisarenko
suit
orchids
tennis
tour
petals
player
Los
ceremony
full
Pirate
Blunt
rose
wore
Crazy
red


In [49]:

def computeMetric(origKeys, predKeys):
    print origKeys
    print predKeys
    n_hits = 0
    n_misses = 0
    n_false_alarms = 0
    for k in predKeys:
        if k in origKeys:
            n_hits += 1
        else:
            n_false_alarms += 1
    for k in origKeys:
        if k not in predKeys:
            n_misses += 1
  
    precision = n_hits*1.0/(n_hits+n_false_alarms)
    recall = n_hits*1.0/(n_hits+n_misses)
  
    F1_score = 2.0*(precision*recall)/(precision+recall)

    print ("Precision = " + str(precision))
    print ("Recall = " + str(recall))
    print ("F1 score = " + str(F1_score))

In [50]:

f = open('Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/train/art_and_culture-20893614.key', 'r')
origKeys = f.read().split("\n")
keys = []
for k in origKeys:
    keys.extend(k.split())

computeMetric(keys, prediction)
# extracting keywords using simple degree centrality
# centrality = nx.degree_centrality(G)
# sorted_centr = sorted(centrality, key=centrality.get, reverse=True)
# predKeys = [nodes_number[i] for i in sorted_centr[:int(0.33*len(sorted_centr))]]
# computeMetric(origKeys, predKeys)
#for i in sorted_centr[:30]:
#    print nodes_number[i]

['married', 'Argentine', 'his', 'Grammy', 'ceremony', 'kisses', 'messages', 'romantically', 'vocal', 'Argentine', 'woman', 'Buble', 'dated', 'thanking', 'wedding', 'ceremony', 'Canadian', 'Michael', 'bouquet', 'vocal', 'album', 'Crazy', 'Love', 'Canadian', 'pop', 'Argentina', 'AP', 'mansion', 'Facebook', 'rose', 'Argentine', 'sweetheart', 'month', 'registry', 'South', 'American', 'after', 'tying', 'downtown', 'beautiful', 'traditional', 'Grammy', 'Vancouver', 'chiffon', 'dress', 'Grammywinning', 'family', 'Lopilato', 'made', 'North', 'bride', 'Argentine', 'TV', 'actress', 'media', 'Twitter', 'met', 'high']
['sitcoms', 'Lopilato', 'sweetheart', 'traditional', 'civil', 'Facebook', 'homes', 'day', 'threw', 'album', 'followers', 'years', 'Grammy', 'dress', 'Buenos', 'woman', 'guests', 'Canada', 'gray', 'star', 'Children', 'top-grossing', 'Monaco', 'British', 'love', 'Girls', 'including', 'BUENOS', 'high', 'wedding', 'pop', 'model', 'several', 'sharp', 'Michael', 'Soul', 'young', 'TV', 'med

In [40]:
len(keys)

58